# UNIDAD DE TRABAJO 1.    
## APTDO. 1.2.4.1 - REDES NEURONALES PREENTRENADAS

### EJEMPLO 1 - Transfer Learning con RESNET50 + TensorFlow/Keras

In [3]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# 📌 1. Cargar el modelo ResNet50 preentrenado (sin la capa de salida)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

2025-02-02 18:04:46.346024: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 204s 2us/step


In [5]:
# 📌 2. Congelar las capas del modelo base (Feature Extraction) 
base_model.trainable = False

In [15]:
# 📌 3. Construir un nuevo modelo agregando capas personalizadas
model = Sequential([
    base_model,
    Flatten(),  
    Dense(256, activation='relu'),
    Dropout(0.5),  
    Dense(2, activation='softmax')  # 2 clases: Gato o Perro
])

In [16]:
# 📌 4. Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [17]:
# 📌 5. Preparar los datos con ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [18]:
train_generator = train_datagen.flow_from_directory(
    "datasets/perros_gatos/",  # 📌 Ruta de imágenes (organizadas en carpetas por clases)
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 800 images belonging to 2 classes.


In [19]:
val_generator = train_datagen.flow_from_directory(
    "datasets/perros_gatos/",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 200 images belonging to 2 classes.


In [20]:
# 📌 6. Entrenar el modelo con Transfer Learning
model.fit(train_generator, validation_data=val_generator, epochs=5)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.5608 - loss: 8.2171 - val_accuracy: 0.5700 - val_loss: 0.7947
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.4850 - loss: 0.9232 - val_accuracy: 0.5100 - val_loss: 0.7046
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.5166 - loss: 0.6892 - val_accuracy: 0.6150 - val_loss: 0.6882
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.5538 - loss: 0.7156 - val_accuracy: 0.7050 - val_loss: 0.6912
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.5648 - loss: 0.6712 - val_accuracy: 0.6050 - val_loss: 0.6892


In [21]:
# 📌 7. Evaluar el modelo en datos de validación
loss, acc = model.evaluate(val_generator)
print(f"Precisión en validación: {acc:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.6255 - loss: 0.7228
Precisión en validación: 0.61


#### Explicación del código   

- Cargamos ResNet50 sin la última capa (include_top=False) para usarlo como extractor de características.  
- Congelamos sus capas (base_model.trainable = False) para que no se ajusten en el nuevo entrenamiento.
- Agregamos nuevas capas personalizadas, adaptadas a la nueva tarea de clasificación (2 clases).
- Compilamos y entrenamos el modelo solo con las nuevas capas.
- Usamos ImageDataGenerator para cargar imágenes desde carpetas y aplicar preprocesamiento.

### EJEMPLO 2 - Transfer Learning con Hugging Face - Sentiment Analysis

#### Importamos librerías   
- **transformers** es una biblioteca de Hugging Face que permite utilizar modelos preentrenados de Procesamiento del Lenguaje Natural (PLN) como BERT, GPT-3, T5, DistilBERT, entre otros.
- **pipeline** es una función de alto nivel que facilita el uso de estos modelos sin necesidad de cargar manualmente los pesos y configuraciones.

In [33]:
# pip install tf-keras
import tensorflow as tf
from transformers import pipeline

#### Creación de modelo preentrenado para análisis de sentimiento   
- Se instancia un pipeline de análisis de sentimiento utilizando un modelo preentrenado por defecto de Hugging Face.
- Si es la primera vez que ejecutas este código, el modelo se descargará automáticamente desde los servidores de Hugging Face.
- Se carga un modelo es multilingüe y puede procesar textos en español. Es el modelo <a href="https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment?utm_source=chatgpt.com"> ***bert-base-multilingual-uncased-sentiment*** </a>desarrollado por NLP Town. Este modelo está basado en la arquitectura BERT y ha sido ajustado específicamente para el análisis de sentimientos en reseñas de productos en seis idiomas: inglés, neerlandés, alemán, francés, español e italiano. Sus características principales son:

  - Arquitectura: Basado en BERT-base-multilingual-uncased, una versión de BERT que maneja múltiples idiomas y no distingue entre mayúsculas y minúsculas.

  - Tarea: Clasificación de sentimientos en reseñas de productos, prediciendo una puntuación en una escala de 1 a 5 estrellas.

In [30]:
# Cargar modelo preentrenado para análisis de sentimiento
# analisis_sentimiento = pipeline("sentiment-analysis")
analisis_sentimiento = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
Device set to use 0


#### Realizar análisis sobre textos dados

In [31]:
# Probar con un texto
# resultado = analisis_sentimiento("This course is very interesting.")
resultado = analisis_sentimiento("Este curso es increíble y muy útil.")
print(resultado)

[{'label': '5 stars', 'score': 0.7860149145126343}]


In [32]:
# Analizar varios textos
textos = ["Me encanta este curso.", "No me gusta la interfaz de usuario.", "Estoy satisfecho con el resultado."]
resultados = analisis_sentimiento(textos)
print(resultados)

[{'label': '5 stars', 'score': 0.7707265615463257}, {'label': '2 stars', 'score': 0.5164674520492554}, {'label': '4 stars', 'score': 0.5573275089263916}]
